# Machine Learning pra Previsão de Evasão Escolar

Neste trabalho será feita uma análise exploratória, limpeza de dados e teste de diversos algoritmos de classificação para prever a evasão escolar.  A acurácia mínima de 90% será buscada.

In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from category_encoders import WOEEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
from imblearn.over_sampling import SMOTE

## Limpeza e transformação dos dados.
Observar, limpar e transformar os dados.
Essa primeira limpeza deve ser feita para que quando o algoritmo receber as novas informações, ele receba apenas as colunas necessárias com os dados corretos.

In [2]:
# Carrega o arquivo em um dataset.  Apaga a coluna que vem com o dataset.
tecnicoJoinville = pd.read_csv("TECNICOS_Joinville5.csv", encoding = 'latin-1', low_memory = False)
tecnicoJoinville.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [3]:
# Observando as 5 primeiras linhas do dataset
tecnicoJoinville.head()

,produto_educacao,carga_horaria,turno,gratuidade_turma,tipo,online,bairro,cidade,genero,escolaridade,situacao_ocupacional,situacao_matricula,IDADE,empresa
0,AUTOMACAO,1200,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Empregado,Concluinte,21,Schulz
1,FABRICACAO,1600,Noite,Mista,PF,Nao,Jardim_Paraiso,Joinville,M,Medio_C,Empregado,Concluinte,18,N_D
2,LOGISTICA,1300,Noite,Mista,PF,Nao,ForaJoinville,SaoFrancisco_Sul,M,Medio_C,Empregado,Concluinte,18,SENAI/SC
3,AUTOMACAO,1600,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Desempregado,Concluinte,16,N_D
4,MECANICA,1620,Tarde,Mista,PF,Nao,Ulysses_Guimaraes,Joinville,M,Medio_C,Empregado,Concluinte,17,Schulz


In [4]:
tecnicoJoinville.shape

(1704, 14)

In [5]:
# Verificando se o dataset está desbalanceado
tecnicoJoinville['situacao_matricula'].value_counts()

Desistente    949
Concluinte    755
Name: situacao_matricula, dtype: int64

In [6]:
tecnicoJoinville = tecnicoJoinville.reset_index(drop = True)

A variável target está desbalanceada. Para que o algoritmo aprenda corretamente é necessário que o dataset esteja balanceado.

## Teste dos algoritmos de classificação

#### Antes de iniciar os testes para escolher o melhor algoritmo, é necessário transformar as variáveis categóricas em variáveis numéricas.
#### Regressão Logística utilizando Ordinal Encoder

In [7]:
# Criando uma lista de nomes dos algoritmos que serão testados. 
# Isso vai permitir criar um dataframe com os resultados de acurácia no final dos testes.
Nomes = ["Regressao", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
Acuracia = []

In [8]:
# Pegando apenas os valores, sem o cabeçalho, em formato de array
data = tecnicoJoinville.values

In [9]:
# Visualizando os dados
data

array([['AUTOMACAO', '1200', 'Tarde', ..., 'Concluinte', 21, 'Schulz'],
       ['FABRICACAO', '1600', 'Noite', ..., 'Concluinte', 18, 'N_D'],
       ['LOGISTICA', '1300', 'Noite', ..., 'Concluinte', 18, 'SENAI/SC'],
       ...,
       ['QUALIDADE', '800', 'Noite', ..., 'Desistente', 23, 'FIBRASCA'],
       ['DESENVOLVIMENTO', '1000', 'Noite', ..., 'Desistente', 20, 'N_D'],
       ['PLASTICO', '1250', 'Manha', ..., 'Desistente', 27, 'LINKPLAS']],
      dtype=object)

In [10]:
# Separando a variável target do dataset:
y = data[:,11]

In [11]:
# Tamanho da variável
y.shape

(1704,)

In [12]:
# Pegando todas as outras colunas
X = data[:,[0,1,2,3,4,5,6,7,8,9,10,12,13]]

In [13]:
# Tamanho do array resultante.
X.shape

(1704, 13)

In [14]:
# Visualizando o array
X

array([['AUTOMACAO', '1200', 'Tarde', ..., 'Empregado', 21, 'Schulz'],
       ['FABRICACAO', '1600', 'Noite', ..., 'Empregado', 18, 'N_D'],
       ['LOGISTICA', '1300', 'Noite', ..., 'Empregado', 18, 'SENAI/SC'],
       ...,
       ['QUALIDADE', '800', 'Noite', ..., 'Empregado', 23, 'FIBRASCA'],
       ['DESENVOLVIMENTO', '1000', 'Noite', ..., 'Empregado', 20, 'N_D'],
       ['PLASTICO', '1250', 'Manha', ..., 'Empregado', 27, 'LINKPLAS']],
      dtype=object)

In [15]:
# Visualizando a variável target
y

array(['Concluinte', 'Concluinte', 'Concluinte', ..., 'Desistente',
       'Desistente', 'Desistente'], dtype=object)

In [16]:
# Aplicando Label Encoder aos dados de saída
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [17]:
# Visualizando a variável target de treino após a aplicação do Label Encoder
y

array([0, 0, 0, ..., 1, 1, 1])

In [18]:
np.all(np.isfinite(y))

True

### Weight of Evidence Encoder

In [19]:
woe_encoder = WOEEncoder(sigma=0.0, regularization=0.0,)
woe_encoder.fit_transform(X = X, y = y)
X = woe_encoder.transform(X)

D:\ANACONDA\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
D:\ANACONDA\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.078794,-0.331220,-0.153890,-0.098879,0.102016,0.851967,0.054671,-0.027634,0.011112,-0.384401,0.016841,0.205347,1.514278
1,0.563547,1.311754,0.054671,-0.098879,0.102016,0.851967,-0.018971,-0.027634,0.011112,-0.384401,0.016841,-0.137719,0.171272
2,0.156971,-0.228691,0.054671,-0.098879,0.102016,0.851967,0.325194,-0.921838,0.011112,-0.384401,0.016841,-0.137719,-0.228691
3,0.078794,1.311754,-0.153890,-0.098879,0.102016,0.851967,0.054671,-0.027634,0.011112,-0.384401,-0.024456,-0.395745,0.171272
4,-0.166171,0.869921,-0.153890,-0.098879,0.102016,0.851967,0.282135,-0.027634,0.011112,-0.384401,0.016841,-0.075722,1.514278
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,-2.479983,inf,0.054671,-0.098879,0.102016,0.851967,-0.451835,-0.027634,0.011112,-0.384401,0.016841,0.134214,inf
1700,inf,2.574669,0.054671,1.405883,0.102016,0.851967,-0.143169,-0.027634,0.011112,-0.384401,0.016841,0.464456,0.171272
1701,inf,2.574669,0.054671,1.405883,0.102016,0.851967,-0.015117,-0.027634,0.011112,-0.384401,0.016841,0.107781,inf
1702,0.955864,0.892394,0.054671,1.405883,0.102016,0.851967,-0.159698,-0.027634,-0.058540,-0.384401,0.016841,0.298329,0.171272


In [21]:
np.all(np.isfinite(X))

False

In [22]:
X[np.isfinite(X) == False] = 0

In [23]:
scaler = StandardScaler(with_mean=False) 
X = X.astype(np.float64)
X = scaler.fit_transform(X)
X = scaler.fit_transform(X)
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [24]:
# Separando em conjunto de treino e teste
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.33, random_state = 0)

In [25]:
X_train1

array([[ 0.83048915,  0.64207281,  0.14667652, ..., -1.19559229,
         0.31705575,  0.25649438],
       [-0.8141771 , -0.45788442, -1.26364703, ...,  0.82332947,
         0.31705575, -1.38053128],
       [-0.63881975,  1.23366222, -0.41286745, ..., -1.19559229,
        -0.18758612,  0.25649438],
       ...,
       [ 0.83048915,  0.64207281,  0.14667652, ...,  0.82332947,
         0.73905399,  0.25649438],
       [-0.8141771 , -0.45788442,  0.14667652, ...,  0.82332947,
        -0.30552793, -0.34248433],
       [-0.19034292, -0.45788442,  0.14667652, ...,  0.82332947,
         0.31705575,  0.25649438]])

In [26]:
#Definindo o modelo
modelo5 = LogisticRegression(solver='saga', max_iter=10000)

In [27]:
modelo5.fit(X_train1, y_train1)
# Previsões nos dados de teste
yhat = modelo5.predict(X_test1)
# Avalia a previsão
accuracy = accuracy_score(y_test1, yhat)
print('Accuracy: %.2f' % (accuracy*100))

# Salvando o resultado da regressão logística com Ordinal Encoder
Acuracia.append(accuracy*100)  ##################################  1

Accuracy: 86.44


## Utilizar o K Nearest Neighbors com Weight of Evidence Encoder

In [28]:
# Cria o modelo
modeloK =  KNeighborsClassifier(30)

In [29]:
# Treina o modelo
modeloK.fit(X_train1, y_train1)
# Fazendo previsões
yhat1 = modeloK.predict(X_test1)
score = modeloK.score(X_test1, y_test1)
# Avaliando as previsões
accuracy = accuracy_score(y_test1, yhat1)
print(score*100)

Acuracia.append(score*100) ######################## 2

85.8054226475279


## Utilizar o SVC com  Weight of Evidence Encoder

In [30]:
# Cria o modelo
modeloSVC = SVC(kernel="linear", C=0.025)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 3

84.37001594896331


## Utilizar o SVC com outros parâmetros, Weight of Evidence Encoder

In [31]:
# Cria o modelo
modeloSVC = SVC(gamma=2, C=1)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 4

77.83094098883574


## Utilizar o classificador Gaussiano, Weight of Evidence Encoder

In [32]:
# Cria o modelo
modeloGaus = GaussianProcessClassifier(1.0 * RBF(1.0))

# Treina o modelo
modeloGaus.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGaus.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

    
Acuracia.append(score*100) ######################## 5

86.76236044657098


## Utilizar o Decision Tree, Weight of Evidence Encoder

In [33]:
# Cria o modelo
modeloDT = DecisionTreeClassifier(max_depth=5)

# Treina o modelo
modeloDT.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloDT.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 6

84.84848484848484


## Utilizar o Random Forest, Weight of Evidence Encoder

In [34]:
# Cria o modelo
modeloRF = RandomForestClassifier(max_depth=50, n_estimators=100, max_features=13)

# Treina o modelo
modeloRF.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloRF.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)
previsoes = modeloRF.predict(X_test1)
Acuracia.append(score*100)  ######################## 7

84.37001594896331


## Utilizar uma rede neural, Weight of Evidence Encoder


In [35]:

# Cria o modelo
modeloMLP = MLPClassifier(alpha=2, max_iter=2000)

# Treina o modelo
modeloMLP.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloMLP.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 8

86.92185007974481


## Utilizar o classificador ADA Boost, Weight of Evidence Encoder

In [36]:
# Cria o modelo
modeloABC =  AdaBoostClassifier()

# Treina o modelo
modeloABC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloABC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 9

85.64593301435407


## Utilizar o classificador Naive Bayes, Weight of Evidence Encoder

In [37]:
# Cria o modelo
modeloGNB =  GaussianNB()

# Treina o modelo
modeloGNB.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGNB.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 10

76.71451355661883


## Utilizar o classificador Quadratic Discriminant Analysis, Weight of Evidence Encoder

In [38]:
# Cria o modelo
modeloQDA =  QuadraticDiscriminantAnalysis()

# Treina o modelo
modeloQDA.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloQDA.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100) ######################## 11

77.19298245614034


In [39]:
Acuracia

[86.44338118022328,
 85.8054226475279,
 84.37001594896331,
 77.83094098883574,
 86.76236044657098,
 84.84848484848484,
 84.37001594896331,
 86.92185007974481,
 85.64593301435407,
 76.71451355661883,
 77.19298245614034]

In [40]:
#Resultados = pd.DataFrame(Nomes, Acuracia)
Resultados = pd.DataFrame(list(zip(Nomes, Acuracia)),columns =['Nomes', 'Acuracia'])

In [41]:
Resultados

,Nomes,Acuracia
0,Regressao,86.443381
1,Nearest Neighbors,85.805423
2,Linear SVM,84.370016
3,RBF SVM,77.830941
4,Gaussian Process,86.762360
5,Decision Tree,84.848485
6,Random Forest,84.370016
7,Neural Net,86.921850
8,AdaBoost,85.645933
9,Naive Bayes,76.714514


In [42]:

print('Modelo escolhido: %.2f' % (Resultados['Acuracia'].max()))

Modelo escolhido: 86.92


In [43]:
modeloMLP

MLPClassifier(alpha=2, max_iter=2000)

In [44]:
# Desistente ==> 1
# Concluinte ==> 0
previsoes = modeloMLP.predict(X_test1)
print(previsoes)

[1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 1
 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0
 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0
 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1
 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1
 1 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1
 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0
 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1
 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0
 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0
 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 1 0 0
 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1
 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0
 0 0 0 1 0 0 1 1 0 1 0 1 

In [45]:
len(previsoes)

627

In [46]:
respFinal = []
for i in range(0,len(previsoes)):
    if previsoes[i] == 1:
        respFinal.append("Desistente")
    else:
        respFinal.append("Concluinte")
respFinal
    

['Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desisten

In [47]:
# save the model to disk
filename = 'ModeloFinal.sav'
pickle.dump(modeloMLP, open(filename, 'wb'))